# NVP para ajustar datos originales y latentes

Se implementa una red neuronal conocida como flujo normalizador que ajustara en los datos originales y latentes una distribucion de probabilidad para modelar la complejidad de ambos espacios de alta dimensión.

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import tensorflow_probability as tfp

2025-12-02 16:37:42.860399: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-02 16:37:42.891322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-02 16:37:42.921160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-02 16:37:42.930186: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-02 16:37:42.953000: I tensorflow/core/platform/cpu_feature_guar

In [4]:
#preprocesamiento de datos
data = pd.read_csv('Liver_GSE14520_U133A.csv')
data.drop(['samples','type'], axis=1, inplace=True)
data.head()

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,6.801198,4.553189,6.787790,5.430893,3.250222,6.272688,3.413405,3.374910,3.654116,3.804983,...,10.735084,10.398843,12.298551,12.270505,3.855588,3.148321,3.366087,3.199008,3.160388,3.366417
1,7.585956,4.193540,3.763183,6.003593,3.309387,6.291927,3.754777,3.587603,5.137159,8.622475,...,11.528447,11.369919,12.867048,12.560433,4.016561,3.282867,3.541994,3.548680,3.460083,3.423348
2,7.803370,4.134075,3.433113,5.395057,3.476944,5.825713,3.505036,3.687333,4.515175,12.681439,...,10.892460,10.416151,12.356337,11.888482,3.839367,3.598851,3.516791,3.484089,3.282626,3.512024
3,6.920840,4.000651,3.754500,5.645297,3.387530,6.470458,3.629249,3.577534,5.192624,11.759412,...,10.686871,10.524836,12.006596,11.846195,3.867602,3.180472,3.309547,3.425501,3.166613,3.377499
4,6.556480,4.599010,4.066155,6.344537,3.372081,5.439280,3.762213,3.440714,4.961625,10.318552,...,11.014454,10.775566,12.657182,12.573076,4.091440,3.306729,3.493704,3.205771,3.378567,3.392938


In [5]:
# Normalización de los datos
data = data.values.astype(np.float32)
data = StandardScaler().fit_transform(data)

In [6]:
#definimos una funcion que tomara un numero aleatorio de caracteristicas por experimento
def get_random_subset(data, n_features, seed=None):
    if seed is not None:
        np.random.seed(seed)
    n_cols = data.shape[1]
    index = np.random.choice(n_cols, size=n_features, replace=False)
    return data[:, index], index

In [7]:
subset1, index1 = get_random_subset(data, n_features=1000, seed=1)
print("Columnas seleccionadas:", index1[:10])


Columnas seleccionadas: [ 1385 19688  4501  4158  2051 20456 17802  3294 19909   415]


In [8]:
subset2, index2 = get_random_subset(data, n_features=2000, seed=2)
print("Columnas seleccionadas:", index2[:10])

Columnas seleccionadas: [20590  5641 18293  2163 13520 21277 15527 19294  9461 15239]


In [28]:
subset3, index3 = get_random_subset(data, n_features=500, seed=3)
print("Columnas seleccionadas:", index3[:10])

Columnas seleccionadas: [ 4833  1353  7457 21932 14801  1927  5673 12350 17548   547]


In [86]:
import wandb
input_dim = subset3.shape
run = wandb.init(project = "Experimentos para NVP",
                 config ={ #configuramos los hiperparametros
                     "learning_rate": 1e-5, 
                     "epochs": 50,
                     "num_blocks": 3,
                     "hidden_units": 100,
                     "input_dim":input_dim,
                     "optimizer": "Adam",
                     "n_features": 500
                 })
config = wandb.config
            
                     
               

epoch,▁▁▁▂▂▃▄▄▄▄▅▅▅▆▆▇▇▇██▁▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▇▇▇█
loss,█▇▃▅▆▃▄▇▆▆▂▄▇▂▆▄▆▆▅█▄▃▂▃▁▂▄▃▆▄▁▆▆▂▁▄▆▄▂▆
epoch,49
loss,709.90039


In [30]:
#generamos un dataset de tensorflow
dataset = tf.data.Dataset.from_tensor_slices(subset3)
dataset = dataset.shuffle(buffer_size=357).batch(57)

In [87]:
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

num_blocks = config.num_blocks
hidden_units = config.hidden_units


class RealNVP(keras.Model):
    def __init__(self, input_dim, num_blocks, hidden_units, base_distribution=None, **kwargs):
        super(RealNVP, self).__init__(**kwargs)
        self.input_dim = input_dim                                                
        self.num_blocks = num_blocks
        self.hidden_units = hidden_units
        self.base_distribution = base_distribution or tfd.MultivariateNormalDiag(  #distribucion normal diagonal de media cero y varianza uno
            loc=tf.zeros(input_dim),
            scale_diag=tf.ones(input_dim)
        )
        self.nll_tracker = keras.metrics.Mean(name="nll")
        # Flujo de z->x cadena de bijectors
        bijectors = []
        self.nets = []
        
        def shift_and_log_scale(x, hidden_units):
            h = tf.keras.layers.Dense(hidden_units, activation="relu")(x)
            h = tf.keras.layers.Dense(hidden_units,activation= "relu")(h)
            h = tf.keras.layers.Dense(hidden_units, activation="relu")(h)
                                    
            

            shift = tf.keras.layers.Dense(x.shape[1], activation="linear")(h)
            log_scale = tf.keras.layers.Dense(x.shape[1], activation="sigmoid")(h)
            return shift, log_scale
            
            for i in range(num_blocks):
                def net(x, hu=hidden_units):
                    return shift_and_log_scale(x, hu)
                    
            bijectors.append(
                tfb.RealNVP(
                    num_masked=input_dim // 2,             #enmascaramos la mitad de las caracteristicas
                    shift_and_log_scale_fn=net,
                    name=f"RealNVP_{i}"
                )
            )

            bijectors.append(                            
                tfb.Permute(
                    permutation=tf.random.shuffle(tf.range(input_dim)),  #se añade una permutación para asegurar que todas las dimensiones interactuen 
                    name=f"Permute_{i}"
                )
            )

        self.flow_bijector = tfb.Chain(list(reversed(bijectors)))
        self.flow_distribution = tfd.TransformedDistribution(            #creamos la distribucion transformada
            distribution=self.base_distribution,
            bijector=self.flow_bijector
        )
        self.trainable_vars = []                                       #los bijectors con el atributo shift_and_log_scale tienen los
        for b in self.flow_distribution.bijector.bijectors:            #parámetros de traslación y escala para las dimensiones transformadas
            if hasattr(b, "shift_and_log_scale_fn") and hasattr(b.shift_and_log_scale_fn, "trainable_variables"):
                self.trainable_vars += b.shift_and_log_scale_fn.trainable_variables


    def call(self, inputs):
            return self.flow_distribution(inputs)  
   
    @property
    def metrics(self):
            return [self.nll_tracker]

    def train_step(self, data):
            with tf.GradientTape() as tape:
                nll = -tf.reduce_mean(self.flow_distribution.log_prob(data)) #verosimilitud de que los datos originales sean observados bajo la distro 
            grads = tape.gradient(nll, self.trainable_vars)
            self.optimizer.apply_gradients(zip(grads, self.trainable_vars))
            self.nll_tracker.update_state(nll)
            return {"loss": nll}
        # Loguear métricas en W&B
            wandb.log({
                "nll": nll.numpy(),
                "grad_norm": grad_norm.numpy()
            })

        
                        

In [88]:
normalizing_flow = RealNVP(config.n_features, config.num_blocks, config.hidden_units)
for batch in dataset.take(1):
    initial_nll = -tf.reduce_mean(normalizing_flow.flow_distribution.log_prob(batch))
    print("NLL inicial (antes de entrenar):", initial_nll.numpy())


NLL inicial (antes de entrenar): 707.7148


2025-12-03 03:07:37.904726: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [89]:
normalizing_flow.compile(optimizer=keras.optimizers.Adam(config.learning_rate))
history = normalizing_flow.fit(dataset,epochs=config.epochs,callbacks=[wandb.keras.WandbCallback()])

Epoch 1/50
7/7 [==============================] - 0s 6ms/step - loss: 702.7773
Epoch 2/50
7/7 [==============================] - 0s 5ms/step - loss: 710.5681
Epoch 3/50
7/7 [==============================] - 0s 5ms/step - loss: 692.8261
Epoch 4/50
7/7 [==============================] - 0s 6ms/step - loss: 701.6533
Epoch 5/50
7/7 [==============================] - 0s 6ms/step - loss: 709.4128
Epoch 6/50
7/7 [==============================] - 0s 6ms/step - loss: 706.6499
Epoch 7/50
7/7 [==============================] - 0s 6ms/step - loss: 712.9219
Epoch 8/50
7/7 [==============================] - 0s 5ms/step - loss: 701.4960
Epoch 9/50
7/7 [==============================] - 0s 6ms/step - loss: 709.8331
Epoch 10/50
7/7 [==============================] - 0s 6ms/step - loss: 711.6962
Epoch 11/50
7/7 [==============================] - 0s 5ms/step - loss: 702.8365
Epoch 12/50
7/7 [==============================] - 0s 5ms/step - loss: 701.7831
Epoch 13/50
7/7 [==============================] 